# Use A CNN To Process The Image

# Requirements 

### Datasets:
Kaggle Data Set:

### Python:
Python3 (3.9.18)

### Modules:
TensorFlow (2.14.0)

Numpy (1.24.3)

Pillow (10.0.1)

The Goal of this will be to build a CNN to determine malignant vs benign tumors based on patient slides. 

To Do List
1) Load the data and Assess the data imaging slides to get the amount of slides as well as how best to proceed
Two options will be pursued, both a down sampling and and up sampling with affine 
2a) down sample to assess items
2b) run augmentations and other transformations as this will enable us get some practice in
one question i have is what is the difference betweem skimage and keras preprocessing
It seems that unless we are running real imaging analysis such as pixel based or edge detection or segemention that its better to run it through keras because it is more efficient at setting up the augmentation transformations. 

Thus I will use keras data preprocessing image and feed it through the keras tensforflow cnn


3) setup these slides so that we have an equal amount of images in both the malignant and the benign set in the training set
4) setup the cnn model

# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
from platform import python_version
print(python_version())
print (tf.__version__)

3.10.13
2.10.0


## Part 1 - Loading The Dataset

In [3]:
dataset_path = 'P:/Portfolio Sets/CNN Breast Tumors/BreaKHis 400X/train' #watch the direction of slashes, '\' will confuse python use '/' or '\\'

### The Training set

In [4]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path, #file path
    labels = 'inferred',  # Automatically infer labels from directory structure (folder names)
    label_mode = 'categorical',  # categorical/int/etc.
    color_mode='rgb', #sets the color of the images rgb/grayscale
    image_size = (64, 64), # Target image size for resizing
    batch_size = 32,  # Batch size for training
    seed = 100, #same random selection instance each time, required for splitting into training and validation sets
    validation_split = 0.2,  # Split the dataset into training and validation sets
    subset = 'training')  # Specify if it's the training subset

train_dataset.class_names

Found 1148 files belonging to 2 classes.
Using 919 files for training.


['benign', 'malignant']

### Validation Set

In [5]:
# Load the validation set
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path, 
    labels = 'inferred',
    label_mode ='categorical',
    color_mode='rgb',
    image_size = (64, 64),
    batch_size = 32,
    seed = 100,
    validation_split = 0.2,
    subset = 'validation', # Specify if it's the validation subset)  
)
validation_dataset.class_names

Found 1148 files belonging to 2 classes.
Using 229 files for validation.


['benign', 'malignant']

## Part 2 - Preprocessing and Augmentation 
Augmentation avoids overfitting the training data.
Apply Augmentation to training set only. Applying only to the training set avoids overfitting.
Transformations are geometric transformations, rotate images, zoom in/out, flip images, etc. This is called augmentation
Augmentation can be applied to the dataset at 1 of 2 different points

Option 1) The Preprocessing Step prior to CNN model (Easier Modularity and Maintenance)

Option 2) Included in the actual CNN model (Easier Deployment)

## Part 3 - Building the CNN

### Convolution and Neural Network

In [6]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation ='relu', input_shape = (64, 64, 3)),
    tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2, padding = 'valid'),
    tf.keras.layers.Flatten(), #last layer before moving data to neural network
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(2, activation = 'softmax')
])

### Compile The CNN

In [7]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Part 4 - Training the CNN

### Training and Evaluating

In [8]:
cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = 15)

Epoch 1/15
29/29 [==============================] - 17s 417ms/step - loss: 179.4126 - accuracy: 0.6605 - val_loss: 2.8058 - val_accuracy: 0.6070
Epoch 2/15
29/29 [==============================] - 3s 97ms/step - loss: 3.6611 - accuracy: 0.6877 - val_loss: 1.5539 - val_accuracy: 0.6900
Epoch 3/15
29/29 [==============================] - 3s 100ms/step - loss: 2.9460 - accuracy: 0.7476 - val_loss: 2.0600 - val_accuracy: 0.8297
Epoch 4/15
29/29 [==============================] - 3s 95ms/step - loss: 2.2461 - accuracy: 0.8074 - val_loss: 1.1065 - val_accuracy: 0.8821
Epoch 5/15
29/29 [==============================] - 3s 96ms/step - loss: 1.2627 - accuracy: 0.8716 - val_loss: 2.2109 - val_accuracy: 0.7336
Epoch 6/15
29/29 [==============================] - 3s 98ms/step - loss: 0.3000 - accuracy: 0.9206 - val_loss: 0.6353 - val_accuracy: 0.8384
Epoch 7/15
29/29 [==============================] - 3s 97ms/step - loss: 0.3403 - accuracy: 0.8803 - val_loss: 1.2536 - val_accuracy: 0.8166
Epoch 8/

## Part 4 - Making a single prediction

import numpy as np
from keras import utils
test_image = utils.load_img('P:\Machine-Learning Course\Machine Learning A-Z (Codes and Datasets)\AG Worksheets\dataset\Single Prediction\dogorcat1.jpg', target_size = (64, 64))
test_image = utils.img_to_array(test_image) #changes to an array to the cnn model can analyze
test_image = np.expand_dims(test_image, axis = 0) #adds an extra dimension to enable the image to have the batch dimension
#batch dimension required because the model has to run at a certain batch number, in this example 32
#dimension of batch is added to the 1st dimension

#we can now run the predict method
result = cnn.predict(test_image)

#to figure out which class is index as what
training_set.class_indices
if result[0][0] == 1: #result 1st slot as batch dimension so we run result[0] to enter the batch the next [0] selects the element in the batch(the single dog image) 
    prediction = 'Dog'
    
else:
    prediction = 'Cat'
print(prediction)